In [75]:
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd
import codecs
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import re
import csv
import string
from datetime import datetime
from collections import Counter
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import   RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
# --------------------------------------- #
import pyarabic.araby as araby
import pyarabic.number as number
from pyarabic.araby import strip_tashkeel
from pyarabic.araby import strip_tatweel
from pyarabic.araby import tokenize, is_arabicrange, strip_tashkeel
# --------------------------------------- #
import pyarabic.arabrepr
arepr = pyarabic.arabrepr.ArabicRepr()
repr = arepr.repr
from tashaphyne.stemming import ArabicLightStemmer
import emoji


import seaborn
import matplotlib.pyplot as plt

%matplotlib inline

[nltk_data] Downloading package punkt to /Users/nadaa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# reading the new data

- note: you may read the data differently , this is an example to show you.

In [79]:
df = pd.read_csv('End_of_campaign_Hashtag.csv' ,names=["created_at", "source", "screen_name", 'followers_count',"location",
                                        "verified",'in_reply_to_screen_name',"in_reply_to_user_id_str","reply_count",
                                        "retweet_count","favorite_count","lang","favorited","retweeted","addition_data","text"])

In [80]:
df

,created_at,source,screen_name,followers_count,location,verified,in_reply_to_screen_name,in_reply_to_user_id_str,reply_count,retweet_count,favorite_count,lang,favorited,retweeted,addition_data,text
0,Tue Nov 24 19:10:31 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",M_World2020,17,المملكة العربية السعودية,False,NaN,NaN,0,0,0,und,False,False,"{'hashtags': [{'text': 'بلاك_فرايدي_شي_إن', 'i...",RT @M_World2020: https://t.co/nyyt4s0bsG\n#بلا...
1,Tue Nov 24 19:08:33 +0000 2020,"<a href=""http://twitter.com/download/iphone"" r...",B7Kgsvd72R2jzNL,22,NaN,False,NaN,NaN,0,0,0,ar,False,False,"{'hashtags': [{'text': 'وفرت_وأنورت', 'indices...",RT @taqa_sa: جمعنا لكم أهم النصائح للتقليل من ...
2,Tue Nov 24 19:00:02 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",Abohedab1,301,NaN,False,NaN,NaN,0,0,0,ar,False,False,"{'hashtags': [{'text': 'وفرت_وأنورت', 'indices...",RT @taqa_sa: الجار قبل الدار والأخضر قبل الأحم...
3,Tue Nov 24 18:42:07 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",Hanoma29441020,69,NaN,False,NaN,NaN,0,0,0,ar,False,False,"{'hashtags': [{'text': 'وفرت_وأنورت', 'indices...",RT @taqa_sa: جمعنا لكم أهم النصائح للتقليل من ...
4,Tue Nov 24 18:41:22 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",560Mohammad,87,NaN,False,NaN,NaN,0,0,0,und,False,False,"{'hashtags': [{'text': 'بلاك_فرايدي_شي_إن', 'i...",RT @M_World2020: https://t.co/nyyt4s0bsG\n#بلا...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8175,Fri Oct 30 08:32:56 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",nooo0011211,555,NaN,False,NaN,NaN,0,0,0,ar,False,False,"{'hashtags': [{'text': 'وفرت_وأنورت', 'indices...",RT @taqa_sa: مع الأخضر عزفك مدوزن ويطرب 🎼\n\n#...
8176,Fri Oct 30 08:32:42 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",44PROFESSOR44,29,⚔️🇸🇦آلمہديہنة آلمہنورة🇸🇦⚔️,False,NaN,NaN,0,0,0,ar,False,False,"{'hashtags': [{'text': 'وفرت_وأنورت', 'indices...",RT @taqa_sa: مع الأخضر عزفك مدوزن ويطرب 🎼\n\n#...
8177,Fri Oct 30 08:30:00 +0000 2020,"<a href=""https://ads.twitter.com"" rel=""nofollo...",taqa_sa,283080,Kingdom of Saudi Arabia,True,NaN,NaN,475,400,4319,ar,False,False,"{'hashtags': [{'text': 'وفرت_وأنورت', 'indices...",مع الأخضر عزفك مدوزن ويطرب 🎼\n\n#وفرت_وأنورت\n...
8178,Thu Oct 29 04:56:16 +0000 2020,"<a href=""http://twitter.com/download/iphone"" r...",ii_B93,240,أسكن المدينة وتسكنني❤️,False,NaN,NaN,1,0,0,ar,False,False,"{'hashtags': [{'text': 'وفرت_وأنورت', 'indices...",#وفرت_وأنورت\nحبّيت الإعلان الأسطوري ❤️❤️ http...


# add id column in the dataset, so it will help us when we merge 

In [81]:
df = df.reset_index()
df["id"] = df.index 
#newdata.head()
df.columns= df.columns.str.lower()
df.columns
df.drop(['index'], axis=1  , inplace=True)

# split the data into 2, data1 for sentiment then will merge with data2 for analysis

- data2 contain ('screen_name','text','id')
- data1 contain all other coulmns + id for merge later

In [83]:
data2 = df[["screen_name",'text','id']] 
data2.head()

,screen_name,text,id
0,M_World2020,RT @M_World2020: https://t.co/nyyt4s0bsG\n#بلا...,0
1,B7Kgsvd72R2jzNL,RT @taqa_sa: جمعنا لكم أهم النصائح للتقليل من ...,1
2,Abohedab1,RT @taqa_sa: الجار قبل الدار والأخضر قبل الأحم...,2
3,Hanoma29441020,RT @taqa_sa: جمعنا لكم أهم النصائح للتقليل من ...,3
4,560Mohammad,RT @M_World2020: https://t.co/nyyt4s0bsG\n#بلا...,4


In [ ]:
data1 = df.copy()
data1.drop(['screen_name', 'text'], axis=1, inplace=True)

# cleaning data2

In [84]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

# def space(text):
#     text= re.sub(r"_", '\s+', s)
#     return text

# new1['text'].replace('_', ' ', regex=True, inplace=True)


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_hashtag(text):
    return text.replace("#\\p{IsAlphabetic}+", "");

In [85]:
def processPost(tweet):

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    #Replace RT with empty string
    tweet = re.sub('RT', ' ', tweet)
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    
    
    #remove non-arabic
#      tweet = re.sub(r'[^ء-ي0-9]',' ',tweet)

    return tweet

In [86]:
data2['text'].replace('[^ء-ي0-9]', ' ', regex=True, inplace=True)

data2['text'].replace('_', ' ', regex=True, inplace=True)

data2['text'] = data2['text'].str.replace(r'\d+','')

data2['text']= data2['text'].apply(lambda x: remove_diacritics(x))

data2['text']= data2['text'].apply(lambda x: remove_repeating_char(x))

data2['text']= data2['text'].apply(lambda x: processPost(x))

data2['text']= data2['text'].apply(lambda x: remove_hashtag(x))

data2['text']= data2['text'].apply(lambda x: remove_punctuations(x))

data2['text']=data2['text'].apply(lambda x: remove_repeating_char(x))

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-86-cce51f8418ad>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['text'] = data2['text'].str.replace(r'\d+','')
<ipython-input-86-cce51f8418ad>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [87]:
tokens = TweetTokenizer()
data2['text'] = data2['text'].apply(tokens.tokenize)

<ipython-input-87-47580ab5b55e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['text'] = data2['text'].apply(tokens.tokenize)


In [88]:
stop = ['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين','و', 'بيد',
         'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك',
         'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما',
         'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك',
         'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا',
         'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا',
         'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير',
         'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها',
         'قد', 'كأن', 'كأنما', 'كأي', 'كأين', 'كذا', 'كذلك', 'كل', 'كلا',
         'كلاهما', 'كلتا', 'كلما', 'كليكما', 'كليهما', 'كم', 'كم', 'كما',
         'كي', 'كيت', 'كيف', 'كيفما', 'لا', 'لاسيما', 'لدى', 'لست', 'لستم',
         'لستما', 'لستن', 'لسن', 'لسنا', 'لعل', 'لك', 'لكم', 'لكما',
         'لكن', 'لكنما', 'لكي', 'لكيلا', 'لم', 'لما', 'لن', 'لنا',
         'له', 'لها', 'لهم', 'لهما', 'لهن', 'لو', 'لولا', 'لوما',
         'لي', 'لئن', 'ليت', 'ليس', 'ليسا', 'ليست', 'ليستا', 'ليسوا', 'ما',
         'ماذا', 'متى', 'مذ', 'مع', 'مما', 'ممن', 'من', 'منه', 'منها', 'منذ',
         'مه', 'مهما', 'نحن', 'نحو', 'نعم', 'ها', 'هاتان', 'هاته', 'هاتي',
         'هاتين', 'هاك', 'هاهنا', 'هذا', 'هذان', 'هذه', 'هذي', 'هذين', 'هكذا',
         'هل', 'هلا', 'هم', 'هما', 'هن', 'هنا', 'هناك', 'هنالك', 'هو', 'هؤلاء',
         'هي', 'هيا', 'هيت', 'هيهات', 'والذي', 'والذين', 'وإذ', 'وإذا', 'وإن',
         'ولا', 'ولكن', 'ولو', 'وما', 'ومن', 'وهو', 'يا' , 'من' , 'على', 'الى','هما', 'مع', 'هذه', 'التي', 'كما ', 'ذلك ', 'لذا', 'عن', 'في','ان','كان','كانت','الى','قبل','أنه','تم'
        ,'وقال','قال','فى','وقد','قد','ولم','وذلك','ذلك','يكون','او','وهذه','وهي ','وبعد','وهذا','عندها','جدا','بأن','انه','الي']

In [89]:
data2['text'] = [w for w in data2['text'] if not w in stop]
data2.head()

<ipython-input-89-76566a917a86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['text'] = [w for w in data2['text'] if not w in stop]


,screen_name,text,id
0,M_World2020,"[بلاك, فرايدي, شي, إن, جامعة, الحفر, عن, بعد, ...",0
1,B7Kgsvd72R2jzNL,"[جمعنا, لكم, أهم, النصائح, لتقليل, من, استهلاك...",1
2,Abohedab1,"[الجار, قبل, الدار, والأخضر, قبل, الأحمر, وفرت...",2
3,Hanoma29441020,"[جمعنا, لكم, أهم, النصائح, لتقليل, من, استهلاك...",3
4,560Mohammad,"[بلاك, فرايدي, شي, إن, جامعة, الحفر, عن, بعد, ...",4


In [92]:
# filter the bad tweets

spam =["حسب الطلب", "للتواصل" , "تنفيذ بالاسماء", "ادخل السحب","مساج","سحوبات"]

def remove_noise(tweet):
    label="ok"
    for word in spam:
        if word in tweet:
            label="spam"

    return label

data2["is_spam"] = data2['text'].apply(lambda x: remove_noise(x))

data2.is_spam.value_counts()

data2=data2[data2["is_spam"]!="spam"]

data2.is_spam.value_counts()

<ipython-input-92-49ccc506c286>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2["is_spam"] = data2['text'].apply(lambda x: remove_noise(x))


ok    8178
Name: is_spam, dtype: int64

# reading the positive lexicon

In [111]:
pos_df=pd.read_csv("pos.list.csv")

In [112]:
pos_list=pos_df["Word"].values.tolist()

In [113]:
len(pos_list)

189

In [114]:
def pos_sentiment(tweet):
    
    count=0
    # loop through the list of words and see if they are in the tweet text
    for word in pos_list:
        
        if word in tweet:
            count+=1
    
    return count

In [115]:
data2["pos_count"] = data2['text'].apply(lambda x: pos_sentiment(x))

# reading the negative lexicon 

In [116]:
neg_df = pd.read_csv("neg.list.csv", 'w', encoding='utf-8-sig')

In [117]:
neg_df.head(5)

,Word
0,الغير اخلاقيه
1,ابحول
2,ابلغ عليكم
3,ابو شكلك
4,ابو كلب


In [118]:
neg_list=neg_df["Word"].values.tolist()

In [119]:
len(neg_list)

777

In [120]:
def neg_sentiment(tweet):
    
    count=0
     # loop through the list of words and see if they are in the tweet text
    for word in neg_list:
        if word in tweet:
            count+=1
               
    return count

In [121]:
data2["neg_count"] = data2['text'].apply(lambda x: neg_sentiment(x))

# New column

In [122]:
def tweet_sentiment(tweet):
    
    pos_sent=pos_sentiment(tweet)
    
    neg_sent=neg_sentiment(tweet)
    
    if pos_sent > neg_sent:
        label = "Positive"
    
    elif pos_sent < neg_sent:
        label = "Negative"
    else:
        label = "Neutral" 
   
   
    return label

In [123]:
data2["sentiment"] = data2['text'].apply(lambda x: tweet_sentiment(x))

# now we have the sentiment column, we drop the 'pos_count', 'neg_count' we don't need it in the analysis

In [124]:
data2.drop(['pos_count', 'neg_count'], axis = 1 , inplace = True)

In [ ]:
data2.head()

# now we join data1 and data2, the rest is all about visualization

In [125]:
# your team can delete unnecessary columns like: tokens, etc...
final_data = data1.merge(data2,on='id')

# codes to help cleaning the dataset

In [ ]:
final_data.duplicated().sum

final_data = final_data.drop_duplicates()

final_data.tail()

final_data.duplicated().sum()

final_data.info()

final_data.tail(3)

final_data.info()

final_data.isnull().sum()

# now we orgnaize the dataset and help in visulaized

In [132]:
final_data.rename({'in_reply_to_screen_name': 'replied_to'}, axis=1, inplace=True)

In [133]:
# this code find thr location

final_data['location'].fillna("Not given", inplace=True)
final_data['replied_to'].fillna("nan", inplace=True)
final_data['in_reply_to_user_id_str'].fillna("nan", inplace=True)
final_data.location.value_counts()

Not given                        4617
المملكة العربية السعودية          876
جدة, المملكة العربية السعودية     234
الرياض                            204
Kingdom of Saudi Arabia           149
                                 ... 
الحدود الشماليه                     1
نجران السعودية                      1
Pakistan                            1
المملكة العربية السعوديه            1
26.479816,50.051769                 1
Name: location, Length: 807, dtype: int64

In [134]:
#find the source 

final_data["source"] = final_data["source"].str.extract('>(.+?)<', expand=False).str.strip() 
final_data.head()

,created_at,source,followers_count,location,verified,replied_to,in_reply_to_user_id_str,reply_count,retweet_count,favorite_count,lang,favorited,retweeted,addition_data,id,screen_name,text,is_spam,sentiment
0,Tue Nov 24 19:10:31 +0000 2020,Twitter for Android,17,المملكة العربية السعودية,False,nan,nan,0,0,0,und,False,False,"{'hashtags': [{'text': 'بلاك_فرايدي_شي_إن', 'i...",0,M_World2020,"[بلاك, فرايدي, شي, إن, جامعة, الحفر, عن, بعد, ...",ok,Neutral
1,Tue Nov 24 19:08:33 +0000 2020,Twitter for iPhone,22,Not given,False,nan,nan,0,0,0,ar,False,False,"{'hashtags': [{'text': 'وفرت_وأنورت', 'indices...",1,B7Kgsvd72R2jzNL,"[جمعنا, لكم, أهم, النصائح, لتقليل, من, استهلاك...",ok,Neutral
2,Tue Nov 24 19:00:02 +0000 2020,Twitter for Android,301,Not given,False,nan,nan,0,0,0,ar,False,False,"{'hashtags': [{'text': 'وفرت_وأنورت', 'indices...",2,Abohedab1,"[الجار, قبل, الدار, والأخضر, قبل, الأحمر, وفرت...",ok,Neutral
3,Tue Nov 24 18:42:07 +0000 2020,Twitter for Android,69,Not given,False,nan,nan,0,0,0,ar,False,False,"{'hashtags': [{'text': 'وفرت_وأنورت', 'indices...",3,Hanoma29441020,"[جمعنا, لكم, أهم, النصائح, لتقليل, من, استهلاك...",ok,Neutral
4,Tue Nov 24 18:41:22 +0000 2020,Twitter for Android,87,Not given,False,nan,nan,0,0,0,und,False,False,"{'hashtags': [{'text': 'بلاك_فرايدي_شي_إن', 'i...",4,560Mohammad,"[بلاك, فرايدي, شي, إن, جامعة, الحفر, عن, بعد, ...",ok,Neutral


In [136]:
#### change to date type, datetime and subset by weeks and days

final_data['created_at'] = pd.to_datetime(final_data.created_at)

import pytz

#all date comes in UTC
#change dattime to saudi arabia 

final_data.created_at = final_data.created_at.dt.tz_convert('Asia/Riyadh')

In [137]:
# change data type and create new columns

final_data['followers_count'] = final_data['followers_count'].astype(int)
final_data['reply_count']= final_data['reply_count'].astype(int)
final_data['retweet_count']= final_data['retweet_count'].astype(int)
final_data['favorite_count']= final_data['favorite_count'].astype(int)


final_data['replied_to']= final_data['replied_to'].astype(str)
final_data['in_reply_to_user_id_str']= final_data['in_reply_to_user_id_str'].astype(str)

final_data['by_hours'] = final_data.created_at.dt.hour
final_data['by_day'] = final_data.created_at.dt.day_name()
final_data['by_month'] = final_data.created_at.dt.month
final_data['by_year'] = final_data.created_at.dt.year

final_data['by_hours'] = final_data['by_hours'].replace([4,5,6,7,8,9,10,11], "Morning")
final_data['by_hours'] = final_data['by_hours'].replace([12,13,14,15,16], "AfterNoon")
final_data['by_hours'] = final_data['by_hours'].replace([17, 18, 19,20,21], "Evening")
final_data['by_hours'] = final_data['by_hours'].replace([22,23,0,1,2,3,], "night")

In [138]:
### after the cleaning find it it is retweet or non retweet

noise=["RT"]

def find_rt(tweet):
    label="Not a Retweet"
    for word in noise:
        if word in tweet:
            label="Retweet"

    return label

final_data["retweet"] = final_data['text'].apply(lambda x: find_rt(x))

In [139]:
# this will find the number of replies to our acc (@taga_sa)

def find_replies(data):
    if data["replied_to"] == "taqa_sa":
        return "Yes"
    elif data["replied_to"] == "nan":
        return "No"
    else:
        return "Yes"


final_data = final_data.assign(replies_to=final_data.apply(find_replies, axis=1))

In [140]:
# more information will be shown as visual

final_data[final_data['replied_to'] == 'taga_sa']

final_data['replied_to'].value_counts()

final_data.replies_to.value_counts()



final_data.retweet.value_counts()

final_data['reply_count'].sum()

final_data['verified'].value_counts()

final_data['replied_to'].value_counts()

final_data.nlargest(3, ['reply_count']) 

final_data.nlargest(10, ['favorite_count']) 

final_data.nlargest(5, ['retweet_count']) 

final_data.source.value_counts().sum()

print('where people tweet from:\n{}'.format(final_data.source.value_counts()))



final_data.by_day.value_counts()

final_data.by_hours.value_counts()


final_data = final_data[final_data['replies_to'] == "Yes"]


final_data['retweeted'].value_counts()

final_data['retweet'].value_counts()

final_data['retweet_count'].sum()

final_data['favorited'].value_counts()

final_data['favorite_count'].sum()

final_data[['favorite_count','retweet_count']].sum()

where people tweet from:
Twitter for iPhone                  4245
Twitter for Android                 3332
Twitter Web App                      360
Twitter for iPad                     151
TweetDeck                             20
Plume for Android                     17
Twitter Ads                           11
Twitter Media Studio                  11
Sprinklr                              10
Twitter for Advertisers                3
Twitter Media Studio - LiveCut         3
Tweetbot for iΟS                       3
Twitter for Advertisers (legacy)       2
moustahlek                             1
sra7h.com                              1
samhnews                               1
Hashflags                              1
WordPress.com                          1
eClincher                              1
XboxInfos                              1
application999999999                   1
IFTTT                                  1
Hootsuite Inc.                         1
Name: source, dtype: int64


favorite_count    334
retweet_count      17
dtype: int64

In [5]:
# find catograize the tweet

acting_and_Ads =["يوسف","الجراح","تمثيل","ممثل", "إعلان", "اعلان", "روعة", "الاعلان","الإعلان", "رهيب", "😂", "😂❤️","❤️", "الحلو", 
        "فنناان","ححلوو", "🤣","💚", "الحلو", "الفنان", "مسلسلاته", "دعاية", "ابداع", "جهودكم","👍","ابداع" ]

compliments = ["👍","🌹","الردود" , "الاحترافية", "جهودكم", "التوعيه", "جميل","يعطيكم","حبيت",
               "شكرا", "شكرا لكم", "احترافي", "تمام","العافية","جنان","حبيتكم","تشكرون"]

complaint = ["الليد","اليد","مافية","مافيه", "السوق","خلصت", "غيرو", "الدقون","مااصدقكم", "حرامية","اسغفر"
            ,"تخرب", 'متوفرة', "متوفره", "مخلصة", "مالقيت","الدقون", "صدقتكم", "لاحول","للاسف","كذب"]

Religion = ["موسيقى", "حرام","الشرع","مايجوز","اسغفر", "لاحول","للاسف","والعياذ"]

def setcategory(tweet):
    label="Others"
    for word in acting:
        if word in tweet:
            label="acting"
            
        for word in compliments:
            if word in tweet:
                label="compliments"
            
            for word in complaint:
                if word in tweet:
                    label="compline"
                    
                for word in ads:
                    if word in tweet:
                        label="ads"
                    
            
    return label

final_data["category"] = final_data['text'].apply(lambda x: setcategory(x))



In [6]:
final_data.head()